In [1]:
from gensim.models import word2vec
from gensim.models import KeyedVectors
import numpy as np
import pandas as pd
import math 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm

In [ ]:
vec_size = 250

sentences = word2vec.LineSentence("WE.txt")
model = word2vec.Word2Vec(sentences, size= vec_size, min_count=1, window=25 ,sg=0 ,iter=400)
model.save("word2vec.model")

In [3]:
model = word2vec.Word2Vec.load("word2vec.model")

In [4]:
model.wv.vectors.shape

(14865, 250)

In [5]:
with open('query_list.txt') as file:
    query_list = file.read().rstrip().split()
with open('doc_list.txt') as file:
    doc_list = file.read().rstrip().split()

query_we = np.zeros((len(query_list),vec_size))
doc_we = np.zeros((len(doc_list),vec_size))

for query_num,query_name in tqdm(enumerate(query_list)):
    with open('Query/' + query_name) as file:
        file = file.read().replace('-1','').rstrip().split()
        for word in file:
            query_we[query_num] += model.wv.__getitem__(str(word)) 
        query_we[query_num] /= len(file)    

for doc_num,doc_name in tqdm(enumerate(doc_list)):     
    with open('Document/' + doc_name) as file:
        for line in range(3):  #we don't want first three line data
            file.readline() 
        file = file.read().replace('-1','').rstrip().split()
        for word in file:
            doc_we[doc_num] += model.wv.__getitem__(str(word)) 
        doc_we[doc_num] /= len(file) 


In [6]:
VSM = cosine_similarity(query_we,doc_we)

In [7]:
VSM = pd.DataFrame(VSM,columns = doc_list ,index = query_list)
VSM 

,VOM19980220.0700.0166,VOM19980220.0700.0221,VOM19980220.0700.0265,VOM19980220.0700.0321,VOM19980220.0700.0359,VOM19980220.0700.0391,VOM19980220.0700.0448,VOM19980220.0700.0487,VOM19980220.0700.0521,VOM19980220.0700.0559,...,VOM19980630.0730.0216,VOM19980630.0730.0248,VOM19980630.0730.0268,VOM19980630.0900.0005,VOM19980630.0900.0040,VOM19980630.0900.0105,VOM19980630.0900.0127,VOM19980630.0900.0169,VOM19980630.0900.0205,VOM19980630.0900.0230
50001.query,-0.159918,-0.229076,-0.230578,-0.006712,-0.123225,-0.032794,-0.107445,-0.077693,0.076480,0.080343,...,-0.167880,-0.088241,-0.010373,0.153009,-0.022899,0.420588,-0.004549,-0.057391,-0.140325,-0.089388
50002.query,0.034846,-0.274032,-0.231997,0.042033,-0.129929,-0.091820,-0.026575,-0.093150,0.006284,0.118041,...,-0.077152,0.066935,-0.043976,-0.184277,-0.274297,-0.170808,-0.151440,-0.006741,-0.203660,-0.273265
50003.query,-0.030210,-0.236078,-0.335563,0.069277,-0.096065,-0.198846,-0.178192,-0.298861,-0.013638,0.093718,...,-0.051911,0.182692,-0.279812,0.149825,-0.312129,0.006610,-0.031604,-0.226971,0.083700,-0.145860
50004.query,-0.108481,-0.120085,-0.194321,0.120623,0.039114,-0.026672,0.001324,-0.066995,0.113938,0.202833,...,0.044669,0.108711,-0.073033,0.680221,0.102853,0.226210,-0.060827,-0.003515,0.092156,0.166478
50005.query,0.056740,-0.238606,-0.373612,-0.223386,-0.280145,-0.173651,-0.155142,-0.126657,-0.061463,-0.076742,...,-0.129355,0.015485,-0.244426,-0.140839,-0.302620,-0.145451,-0.196387,-0.241495,-0.191836,-0.279336
50006.query,-0.113360,-0.186149,-0.216643,0.097079,-0.029540,-0.061818,-0.049920,-0.073060,0.097477,0.129878,...,-0.078924,0.146663,-0.081206,0.207336,-0.046101,0.439354,0.055156,-0.079681,-0.002167,-0.071284
50007.query,-0.152716,-0.158298,-0.189368,0.071342,0.053230,-0.095124,-0.136455,-0.131195,0.080640,0.103965,...,-0.046084,0.193213,-0.097070,0.368059,0.046988,0.485169,0.034550,-0.052412,0.056923,0.073692
50008.query,-0.067011,-0.123021,-0.233308,0.103613,-0.015588,-0.130181,-0.241694,-0.266894,0.079566,0.171853,...,-0.120807,0.012639,-0.224822,0.339630,-0.174263,0.143382,-0.077206,-0.119844,0.026453,0.030339
50009.query,-0.116454,-0.289626,-0.281979,0.031145,0.004822,-0.144020,-0.195459,-0.337216,0.026078,0.119334,...,-0.212924,0.060036,-0.177366,0.343217,-0.102172,0.201981,-0.210852,-0.133350,-0.050550,0.013664
50010.query,-0.010614,-0.260144,-0.399441,0.075248,0.013238,-0.192850,-0.063805,-0.229533,0.004547,0.079753,...,-0.051556,0.218120,-0.237506,0.073460,-0.228343,0.013828,-0.125610,-0.260310,0.010176,-0.112870


In [8]:
f = open('wordembadding.txt','w')    #write in txt file
f.write('Query,RetrievedDocuments\n')
for i in range(len(query_list)):
    f.write(VSM.index[i])
    f.write(',')
    VSM = VSM.sort_values(by = query_list[i],ascending= False,axis = 1)
    for j in range(len(doc_list)): #this time evaluate jusy map@50
        f.write(VSM.columns[j])
        f.write(' ')
    f.write('\n')